# Covered Call

- With rollover
- Under different market condition, include

# 策略比較

Covered Call vs 持有正股？

- 加槓桿

Cover Call vs Long Call

- 皆持有正股的情況
- Long Call
  - 優：具有上漲時加倉、下跌時減倉的特性，
  - 缺：有 time decay、需要付出額外成本（edge）

# Market condition

Monte Carlo

# Search Covered Call hyper params

- Rollover criteria: expiry, delta
- Leverage

Dynamic Rollover

- 加減倉：上漲時加倉、下跌時減倉
  - 指標：RSI, IV, delta...？
- 可以是 discrete，不一定 continuous
  - eg 當 delta 一日變動值大於某個幅度 -> 減倉

# 策略

delta 控制在 0.3-0.7，超過則 rollover， roll 標的選到期日小於 1 個月，delta 0.5

1. 正股

   - 上漲時/下跌時：1. 加倉 2. 減倉
   - 大漲時/大跌時：1. 加倉 2. 減倉

2. Option Rollover

   - delta > 0.7 or 到離 1 週前 (delta < 0.3 操作類似)

   1. 傾向做止損, roll 至 0.6 （ATM）
   2. 傾向做加碼, roll 至 0.4 （OTM）
   3. 持平 roll 至 0.5

   - 大跌時

   1. 選擇權平倉
   2. 選擇權維持原策略


In [ ]:
"""
Plot BSM option price vs strike price
"""

import os
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

from option_backtest.gen_data import generate_and_save_stock_data, generate_options
from option_backtest.market import Account, Option, Stock, StrategyFn, Trade, episode
from option_backtest.option import FindOptionsBy, find_options
from option_backtest.bt_event import (
    ShouldRolloverCriteria,
    calculate_metrics,
    create_rollover_strategy,
    plot_results,
)


def demo(strategy_fn: StrategyFn):
    # Read or generate stock data
    stock_id = "XYZ"
    output_dir = f"output/{stock_id}"

    s0 = 100.0  # 初始價格
    mu = 0.05  # 預期收益率（年化）
    sigma = 0.2  # 波動率（年化）
    dt = 1 / 252  # 時間步長（假設一年有252個交易日）
    n_steps = 252  # 模擬步數（假設模擬一年的價格路徑）
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)

    stock_price_path = f"{output_dir}/{stock_id}_price.csv"
    if not os.path.exists(stock_price_path):
        generate_and_save_stock_data(
            s0, mu, sigma, dt, start_date, end_date, stock_price_path
        )

    stock_price_df = pd.read_csv(stock_price_path)
    stock_price_df["date"] = pd.to_datetime(stock_price_df["date"])
    stock_price_df = stock_price_df.set_index("date")
    stock = Stock(id="XYZ", df=stock_price_df)

    # Generate options data
    r = 0.05  # Risk-free rate
    sigma_option = 0.2  # Volatility

    strikes = np.arange(
        (min(stock.df["price"]) // 5 - 2) * 5,
        (max(stock.df["price"]) // 5 + 3) * 5 + 1,
        5,
    )
    expiry_dates = list(pd.date_range("2023-01-01", periods=13, freq="M"))
    listed_options = generate_options(
        stock,
        strikes,
        expiry_dates,
        r,
        sigma_option,
    )

    # Run backtest
    init_cash = 1e3
    trades, portfolio_df = episode(init_cash, stock, listed_options, strategy_fn)

    # Calculate metrics
    metrics = calculate_metrics(portfolio_df)

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

    # Plot results
    plot_results(stock.df, listed_options, portfolio_df, trades)

In [ ]:
# Strtegy Static Roll: Long call, rollover before 2 days, roll to ATM, 30 days+
n_option_legs = 1
entry_position = 1
find_options_by: FindOptionsBy = {
    "option_type": "call",
    "dte_gt": 29,
    "dte_lt": 61,
    "sort_by_delta_near": 0.5,
}
rollover_criteria: ShouldRolloverCriteria = {"dte_lt": 2}

strategy_static_roll = create_rollover_strategy(
    n_option_legs,
    entry_position,
    find_options_by,
    rollover_criteria,
)
# demo(strategy_static_long_call)

In [ ]:
# Strtegy Dynamic Roll
n_option_legs = 1
entry_position = 1
find_options_by: FindOptionsBy = {
    "option_type": "call",
    "dte_gt": 29,
    "dte_lt": 61,
    "sort_by_delta_near": 0.5,
}
rollover_criteria: ShouldRolloverCriteria = {"dte_lt": 2}

strategy = create_rollover_strategy(
    n_option_legs,
    entry_position,
    find_options_by,
    rollover_criteria,
)
demo(strategy)